## Goals

In this guided project, we'll work with data scraped from `Wikipedia`. Volunteer content contributors and editors maintain Wikipedia by continuously improving content. 

We'll implement a simplified version of the `grep` command-line utility to search for data in `54` megabytes worth of articles. If you're not familiar with the grep command, the grep utility essentially allows searching for textual data in all files from a given directory.

- Search for all occurrences of a string in all of the files.
- Provide a case-insensitive option to the search.
- Refine the result by providing the specific locations of the files.

## Data

Articles were saved using the last component of their URLs. For example, a page on Wikipedia has the URL structure https://en.wikipedia.org/wiki/Yarkant_County. If we were saving the article with the previous URL, we'd save it to the file Yarkant_County.html. All the data files are in the wiki folder. Note that the files are raw HTML — We don't need to understand HTML for this guided project. We're going to treat those files like plain-text and we won't rely on any of the specific structure of those files.

# 1. Introducing to Wikipedia Data


In [1]:
# import libraries
import os
import math
import functools
from multiprocessing import Pool

In [2]:
# List all of the files in the wiki folder.

folder_name = "wiki"
file_names = os.listdir(folder_name)

# Count and display the number of files in the wiki folder.

n_files = len(file_names)
print(n_files)

999


In [3]:
# Read the first file in the wiki folder, and print its contents.
with open(os.path.join(folder_name, file_names[0])) as f:
    lines= [line for line in f.readlines()]

# 2. Adding the MapReduce Framework

In [4]:
def make_chunks(data, num_chunks):
    chunk_size = math.ceil(len(data) / num_chunks)
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

def map_reduce(data, num_processes, mapper, reducer):
    chunks = make_chunks(data, num_processes)
    with Pool(num_processes) as pool:
        chunk_results = pool.map(mapper, chunks)
    return functools.reduce(reducer, chunk_results)

# Count the total number of lines in all files. 

Bonus: Use MapReduce for this step.

In [5]:
def map_line_count(file_names):
    total = 0
    for fn in file_names:
        with open(os.path.join(folder_name, fn)) as f:
            total += len(f.readlines())
    return total

def reduce_line_count(count1, count2):
    return count1 + count2

map_reduce(file_names, 8, map_line_count, reduce_line_count)

499797

# 3. Grep Exact Match
- Use MapReduce to create a function that, given a string, creates a dictionary where the keys are the file names and the values are lists with all line indexes that contain the given string.

- Use the function to find all occurrences of the string `"data"` in the files stored in the `wiki` folder.

In [6]:
# The target variable is defined outside and contains the string 
def map_grep(file_names):
    grep = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line for line in f.readlines()]
        for idx, line in enumerate(lines):
            if target in line:
                if fn not in grep:
                    grep[fn] = []
                grep[fn].append(idx)
    return grep

def reduce_grep(line1, line2):
    line1.update(line2)
    return line1

def mapreduce_grep(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes, map_grep, reduce_grep)


In [7]:
target = "data"
data_occurances = mapreduce_grep("wiki", 8)
len(data_occurances)

999

# 4. Grep Case Insensitive


In [8]:
def map_grep_insensitive(file_names):
    grep = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for idx, line in enumerate(lines):
            if target.lower() in line:
                if fn not in grep:
                    grep[fn] = []
                grep[fn].append(idx)
    return grep

def mapreduce_grep_insensitive(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes, map_grep_insensitive, reduce_grep)

In [9]:
target = "data"
data_occurances_case_insensitive = mapreduce_grep_insensitive("wiki", 8)
len(data_occurances_case_insensitive)

999

# 5. Checking the Implementation

Let's verify that the new implementation works by seeing if it finds more matches than the previous implementation.

- Store the results of the search for the string "data" with both versions of the algorithms into variables.

- For each file, if there are more matches in the case insensitive result, print the file name new together with the number of new matches.

In [10]:
for fn in data_occurances_case_insensitive:
    if fn not in data_occurances:
        print("Found {} new matches on file {}".format(len(data_occurances_case_insensitive[fn]), fn))
    elif len(data_occurances_case_insensitive[fn]) > len(data_occurances[fn]):
        print("Found {} new matches on file {}".format(len(data_occurances_case_insensitive[fn]) - len(data_occurances[fn]), fn))

Found 1 new matches on file wiki/Table_Point_Formation.html
Found 1 new matches on file wiki/Ingrid_GuimarC3A3es.html
Found 2 new matches on file wiki/Jules_Verne_ATV.html
Found 1 new matches on file wiki/Pictogram.html
Found 2 new matches on file wiki/Claire_Danes.html
Found 1 new matches on file wiki/PTPRS.html
Found 1 new matches on file wiki/A_Beautiful_Valley.html
Found 1 new matches on file wiki/Mudramothiram.html
Found 2 new matches on file wiki/Gordon_Bau.html
Found 1 new matches on file wiki/Embraer_Unidade_GaviC3A3o_Peixoto_Airport.html
Found 3 new matches on file wiki/Code_page_1023.html
Found 1 new matches on file wiki/Cryptographic_primitive.html
Found 1 new matches on file wiki/Alex_Kurtzman.html
Found 1 new matches on file wiki/Filip_Pyrochta.html
Found 1 new matches on file wiki/Morgana_King.html
Found 1 new matches on file wiki/Don_Parsons_(ice_hockey).html
Found 1 new matches on file wiki/Bias.html
Found 2 new matches on file wiki/Tomohiko_ItC58D_(director).html
Found

# 6. Finding Match Positions on Lines

Right now, we are only finding the line numbers where there is at least one occurrence. Let's extend the algorithm so that it provides information about the location of the matches in those lines.

- Modify the mapper function so that it returns a list of pairs with all occurrences of the target on all files from the given chunk. In each pair, the first number should be the line index and the second number should be the index on the line.

In [11]:
def find_match_indexes(line, target):
    results = []
    i = line.find(target)
    while i != -1:
        results.append(i)
        i = line.find(target, i + 1)
    return results

# Test implementation
s = "Data science is related to data mining, machine learning and big data.".lower()
print(find_match_indexes(s, "data"))

[0, 27, 65]


In [12]:
def map_grep_insensitive_position(file_names):
    grep = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for idx, line in enumerate(lines):
            if target.lower() in line:
                if fn not in grep:
                    grep[fn] = []
                grep[fn] += [(idx, match_idx) for match_idx in find_match_indexes(line, target.lower())]
    return grep
  
def mapreduce_grep_insensitive_position(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes, map_grep_insensitive_position, reduce_grep)

In [13]:
target = "science"
data_occurances_case_insensitive_position = mapreduce_grep_insensitive_position("wiki", 8)
len(data_occurances_case_insensitive_position)

119

In [14]:
data_occurances_case_insensitive_position

{'wiki/Valentin_Yanin.html': [(6, 840),
  (6, 890),
  (66, 90),
  (66, 145),
  (66, 173),
  (144, 1440),
  (144, 1502),
  (144, 1548),
  (144, 1632),
  (144, 1697),
  (144, 1746)],
 'wiki/William_Harvey_Lillard.html': [(80, 166)],
 'wiki/Victor_S._Mamatey.html': [(48, 682), (48, 728), (48, 767)],
 'wiki/Table_Point_Formation.html': [(68, 907), (68, 937), (68, 953)],
 'wiki/Master_of_Space_and_Time.html': [(6, 499),
  (6, 554),
  (45, 531),
  (61, 49),
  (61, 99),
  (61, 122),
  (109, 342),
  (109, 391),
  (109, 424),
  (109, 609),
  (109, 660),
  (109, 695)],
 'wiki/Urban_chicken.html': [(105, 256)],
 'wiki/AlMidan.html': [(277, 48),
  (277, 108),
  (277, 161),
  (281, 48),
  (281, 128),
  (281, 181)],
 'wiki/Jules_Verne_ATV.html': [(208, 507),
  (208, 551),
  (208, 568),
  (427, 231),
  (427, 427),
  (427, 831),
  (427, 971),
  (941, 60),
  (941, 127),
  (982, 29),
  (982, 63),
  (982, 90),
  (1007, 33),
  (1033, 43)],
 'wiki/Pictogram.html': [(491, 47),
  (491, 92),
  (491, 114),
  (

# 7. Displaying the Results

Our grep algorithms can now find all matches. However, with the dictionary it produces, it's not very easy to see those matches.

Let's write the results into a CSV file. Using four columns:

- `File`: shows the name of the file of the match.
- `Line`: shows the index of the line of the match.
- `Index`: shows the index on the line of the match.
- `Context`: shows the text around the match so that users can see the context.

In [15]:
import pandas as pd
import csv

context_delta = 30

with open("results.csv", "w") as f:
    writer = csv.writer(f)
    rows = [["File", "Line", "Index", "Context"]]
    for fn in data_occurances_case_insensitive_position:
        with open(fn) as f:
            lines = [line.strip() for line in f.readlines()]
            for line, idx in data_occurances_case_insensitive_position[fn]:
                start = max(idx - context_delta, 0)
                end = idx + len(target) + context_delta
                rows.append([fn, line, idx, lines[line][start:end]])
        writer.writerows(rows)

In [16]:
df = pd.read_csv("results.csv")
df.head(10)

,File,Line,Index,Context
0,wiki/Valentin_Yanin.html,6,840,"embers of the USSR Academy of Sciences"",""Full ..."
1,wiki/Valentin_Yanin.html,6,890,"ers of the Russian Academy of Sciences"",""Demid..."
2,wiki/Valentin_Yanin.html,66,90,"href=""/wiki/Soviet_Academy_of_Sciences"" class=..."
3,wiki/Valentin_Yanin.html,66,145,"ect"" title=""Soviet Academy of Sciences"">Soviet..."
4,wiki/Valentin_Yanin.html,66,173,"f Sciences"">Soviet Academy of Sciences</a>; he..."
5,wiki/Valentin_Yanin.html,144,1440,"rs_of_the_USSR_Academy_of_Sciences"" title=""Cat..."
6,wiki/Valentin_Yanin.html,144,1502,"rs of the USSR Academy of Sciences"">Full Membe..."
7,wiki/Valentin_Yanin.html,144,1548,rs of the USSR Academy of Sciences</a></li><li...
8,wiki/Valentin_Yanin.html,144,1632,"of_the_Russian_Academy_of_Sciences"" title=""Cat..."
9,wiki/Valentin_Yanin.html,144,1697,"of the Russian Academy of Sciences"">Full Membe..."
